In [2]:
from sklearn.model_selection import KFold
from tensorflow.keras.applications import VGG16, MobileNetV2
from tensorflow.keras import layers, models, Model
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools

# ---- Load and Preprocess Data ----
train_dir = 'D:/Lung_cancer/train'  # Training directory path
target_size = (224, 224)  # Input size for MobileNet-VGG
num_classes = 3

# Extract file paths and labels
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=32,  # Temporary batch size
    class_mode='categorical',
    shuffle=False  # To keep track of indices
)
X = np.array(train_generator.filepaths)  # File paths of images
y = np.array(train_generator.classes)    # Corresponding class labels

# ---- Hyperparameter Search Space ----
batch_sizes = [16, 32]
dropout_rates = [0.3, 0.5]
learning_rates = [0.0005, 0.001]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_params = {}
best_accuracy = 0

# Iterate over all combinations of hyperparameters
for batch_size, dropout_rate, learning_rate in itertools.product(batch_sizes, dropout_rates, learning_rates):
    print(f"Testing batch_size={batch_size}, dropout={dropout_rate}, lr={learning_rate}")
    
    fold_accuracies = []
    fold_losses = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\nFold {fold + 1}")
        
        # Split into training and validation sets
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Load data dynamically
        def load_data(file_paths, labels):
            images = []
            for file in file_paths:
                img = load_img(file, target_size=target_size)
                img = img_to_array(img)
                images.append(img)
            images = np.array(images) / 255.0
            labels = np.eye(num_classes)[labels]
            return images, labels
        
        X_train_images, y_train_labels = load_data(X_train, y_train)
        X_val_images, y_val_labels = load_data(X_val, y_val)
        
        # ---- Define Hybrid MobileNet-VGG Model ----
        mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

        mobilenet_base.trainable = False
        vgg_base.trainable = False

        # Extract features
        mobilenet_features = mobilenet_base.output
        vgg_features = vgg_base.output

        # Merge both extracted features
        combined = layers.Concatenate()([layers.GlobalAveragePooling2D()(mobilenet_features), 
                                         layers.GlobalAveragePooling2D()(vgg_features)])

        x = layers.Dense(512, activation='relu')(combined)
        x = layers.Dropout(dropout_rate)(x)
        output = layers.Dense(num_classes, activation='softmax')(x)
        
        model = Model(inputs=[mobilenet_base.input, vgg_base.input], outputs=output)

        model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # ---- Train Model ----
        history = model.fit(
            [X_train_images, X_train_images], y_train_labels,
            validation_data=([X_val_images, X_val_images], y_val_labels),
            epochs=10,
            batch_size=batch_size,
            verbose=1
        )
        
        # ---- Evaluate Model ----
        val_loss, val_accuracy = model.evaluate([X_val_images, X_val_images], y_val_labels, verbose=0)
        print(f"Fold {fold + 1} Accuracy: {val_accuracy * 100:.2f}%")
        print(f"Fold {fold + 1} Loss: {val_loss:.4f}")
        
        fold_accuracies.append(val_accuracy)
        fold_losses.append(val_loss)
    
    # Compute mean accuracy for the current hyperparameter combination
    mean_accuracy = np.mean(fold_accuracies)
    mean_loss = np.mean(fold_losses)
    
    print(f"Mean Accuracy: {mean_accuracy * 100:.2f}%, Mean Loss: {mean_loss:.4f}\n")
    
    # Store best hyperparameters
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_params = {
            "batch_size": batch_size,
            "dropout": dropout_rate,
            "learning_rate": learning_rate,
            "accuracy": best_accuracy
        }

# ---- Final Best Parameters ----
print("Best Parameters:")
print(best_params)


Found 406 images belonging to 3 classes.
Testing batch_size=16, dropout=0.3, lr=0.0005

Fold 1
Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.4496 - loss: 1.2181 - val_accuracy: 0.6707 - val_loss: 0.7771
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.6677 - loss: 0.8128 - val_accuracy: 0.6098 - val_loss: 0.7801
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.7877 - loss: 0.5113 - val_accuracy: 0.7927 - val_loss: 0.5244
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8122 - loss: 0.4224 - val_accuracy: 0.8171 - val_loss: 0.4898
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.8657 - loss: 0.3473 - val_accuracy: 0.7195 - val_loss: 0.5842
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.9231 - loss: 0.2336 - val_accuracy: 0.8049 - val_loss: 0.4589
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.9429 - loss: 0.1955 - val_accuracy: 0.7927 - val_loss: 0.4696
Epoch 8/10
21/21 ━━━━━━